# 1. **SETUP AND LOGIN TO HF🤗**

In [28]:
!pip install transformers datasets huggingface_hub

In [29]:
from huggingface_hub import notebook_login

notebook_login()

# 2. **USE THE MODELS🤖**

In [30]:
# Cell 1: (re)load the single-model pipelines
import pandas as pd
from collections import Counter
from IPython.display import display
from huggingface_hub import snapshot_download
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline

single_model_repos = {
    "BERT":    "wakaflocka17/bert-imdb-finetuned",
    "BART":    "wakaflocka17/bart-imdb-finetuned",
    "GPT-Neo": "wakaflocka17/gptneo-imdb-finetuned",
}

pipelines = {}
for name, repo_id in single_model_repos.items():
    local_dir = snapshot_download(repo_id)
    tok = AutoTokenizer.from_pretrained(local_dir)
    mdl = AutoModelForSequenceClassification.from_pretrained(local_dir)
    pipelines[name] = pipeline("text-classification", model=mdl, tokenizer=tok, return_all_scores=False)

print("✅ Pipelines loaded.")

# Cell 2: build and display the summary table
reviews = [
    "I absolutely loved this movie!",
    "Quite boring and too long."
]
ground_truths = ["POSITIVE", "NEGATIVE"]

rows = []
for text, gt in zip(reviews, ground_truths):
    preds = {name: pipelines[name]([text], batch_size=1)[0]["label"] for name in pipelines}
    vote = Counter(preds.values()).most_common(1)[0][0]
    rows.append({
        "Review": text,
        "Ground Truth": gt,
        **preds,
        "Ensemble Vote": vote
    })

df = pd.DataFrame(rows)
display(df)

Fetching 10 files:   0%|          | 0/10 [00:00<?, ?it/s]

Device set to use cpu
/usr/local/lib/python3.11/dist-packages/transformers/pipelines/text_classification.py:106: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


Fetching 11 files:   0%|          | 0/11 [00:00<?, ?it/s]

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'LABEL_0', '1': 'LABEL_1'}. The number of labels will be overwritten to 2.
Device set to use cpu


Fetching 14 files:   0%|          | 0/14 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Device set to use cpu


✅ Pipelines loaded.


,Review,Ground Truth,BERT,BART,GPT-Neo,Ensemble Vote
0,I absolutely loved this movie!,POSITIVE,LABEL_1,LABEL_1,LABEL_1,LABEL_1
1,Quite boring and too long.,NEGATIVE,LABEL_0,LABEL_0,LABEL_0,LABEL_0
